In [34]:
import re
import json

def compile(source):
    !bash -lc "cd .. && make -s build/{source}.bench"
    
def bench(source, n, t):
    compile(source)
    res = !./../build/{source}.bench {n} {t}
    return str(res)[1]

def perf(source, n, t):

    PERF_EVENTS = {
        "cycles",
        "instructions",
        "L1-dcache-loads",
        "L1-dcache-load-misses",
        "LLC-loads",
        "LLC-load-misses",
        "cache-references",
        "cache-misses",
    }
    
    compile(source)
    lines = !bash -lc "taskset -c 8 perf stat -r 10 -e cycles,instructions,L1-dcache-loads,L1-dcache-load-misses,LLC-loads,LLC-load-misses,cache-references,cache-misses ./../build/{source}.bench {n} {t}"
    
    data = {}
    for line in lines:
        line = line.split("#", 1)[0]
        m = re.match(r"\s*([\d,]+)\s+([\w\-]+)", line)
        if not m:
            continue

        value = int(m.group(1).replace(",", ""))
        event = m.group(2)

        if event in PERF_EVENTS:
            data[event] = value
    
    return data

In [35]:
perf("matmul_openblas", 1024, 2**32)

{'cycles': 6060907844,
 'instructions': 11227818257,
 'L1-dcache-loads': 4259617037,
 'L1-dcache-load-misses': 761217037,
 'cache-references': 1460962034,
 'cache-misses': 8926631}

In [36]:
perf("matmul12_avx512", 1024, 2**32)

{'cycles': 6075709629,
 'instructions': 12808806596,
 'L1-dcache-loads': 7982927245,
 'L1-dcache-load-misses': 1990496785,
 'cache-references': 4987496615,
 'cache-misses': 43090137}